In [ ]:
#### IMPORTS ####
using ProgressMeter
using Plots 
using Measures
using LaTeXStrings
using DelimitedFiles
# using NPZ
include("source/mOrbRG.jl")
include("source/probes.jl")
const SIZE = (750,300)
scale = 1.5
discreteCgrad = cgrad(:BuPu_3, 3, categorical = true, rev=true)
animName(orbital, num_kspace, scale, W_by_J) = "kondoUb_kspaceRG_$(orbital)_wave_$(num_kspace)_$(round(W_by_J, digits=3))_$(SIZE[1] * scale)x$(SIZE[2] * scale)"

In [ ]:
function getRgFlow(num_kspace_half, J_val, W_by_J_range, orbitals)
    num_kspace = 2 * num_kspace_half + 1
    k_vals = collect(range(K_MIN, K_MAX, length=num_kspace)) ./ pi
    results1_arr = zeros((length(W_by_J_range), num_kspace^2))
    results2_arr = zeros((length(W_by_J_range), num_kspace^2))
    for (j, W_by_J) in collect(enumerate(W_by_J_range))
        savePath = "data/$(orbitals[1])_$(num_kspace_half)_$(round(J_val, digits=3))_$(round(W_by_J, digits=3))"
        kondoJArray, dispersionArray = main(num_kspace_half, J_val, J_val * W_by_J, orbitals[1], orbitals[2])
        results_norm, results_unnorm, results_bool = totalScatterProb(kondoJArray, num_kspace_half+1, savePath)
        results1_arr[j,:] = results_bool
        results2_arr[j,:] = results_unnorm
    end
    max_r = maximum(log10.(results2_arr))
    min_r = minimum(log10.(results2_arr))
    pdfFileNames = []

    anim = @animate for (j, W_by_J) in enumerate(W_by_J_range)
        println()
        hm1 = heatmap(k_vals, k_vals, reshape(results1_arr[j,:], (num_kspace, num_kspace)), 
            cmap=discreteCgrad)
        hm2 = heatmap(k_vals, k_vals, reshape(log10.(results2_arr[j,:]), (num_kspace, num_kspace)),
            cmap=:matter, clims=(min_r, max_r))
        p = plot(hm1, hm2, size=SIZE, 
            plot_title="\$W/J=$(round(W_by_J, digits=3))\$", 
            top_margin=3mm, left_margin=[5mm 10mm], bottom_margin=5mm,
            xlabel="\$ \\mathrm{k_x/\\pi} \$", ylabel="\$ \\mathrm{k_y/\\pi} \$",
            dpi=100 * scale
        )
        pdfFileName = "fig_$(j).pdf"
        savefig(pdfFileName)
        push!(pdfFileNames, pdfFileName)
    end
    run(`pdfunite $pdfFileNames fig_combined.pdf`)
    run(`rm $pdfFileNames`)
    gif(anim, animName(orbitals[1], num_kspace, scale, 1) * ".gif", fps=0.5)
    gif(anim, animName(orbitals[1], num_kspace, scale, 1) * ".mp4", fps=0.5);
end

getRgFlow(10, 0.1, 0:-0.02:-0.06, ["p", "p"])

In [ ]:
LOAD_PATH